# DEA Opioid Data

- Downloaded from here on July 30, 2021 as gzipped .tsv: https://d2ty8gaf6rmowa.cloudfront.net/dea-pain-pill-database/bulk/arcos_all_washpost.tsv.gz (clicking starts download)
- Extracted and uploaded to **coiled-datasets** S3 bucket


In [1]:
import coiled
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
cluster = coiled.Cluster(
    name="data-import",
    software='rrpelgrim/coiled-base-m1',
    n_workers=20,
    worker_cpu=4,
    worker_memory='32Gb',
    scheduler_cpu=2,
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'1hour'},
)

Output()

Found software environment build
Created FW rules: coiled-dask-rrpelgr71-34315-firewall
Created scheduler VM: coiled-dask-rrpelgr71-34315-scheduler (ip: ['54.204.88.218'])


In [3]:
client = Client(cluster)
client

<Client: 'tls://10.4.0.230:8786' processes=20 threads=80, memory=580.00 GiB>

In [11]:
client.restart()

<Client: 'tls://10.4.0.230:8786' processes=19 threads=76, memory=551.00 GiB>

In [5]:
columns_selection = [
 'REPORTER_DEA_NO',
 'REPORTER_BUS_ACT',
 'REPORTER_NAME',
 'REPORTER_ADDL_CO_INFO',
 'REPORTER_ADDRESS1',
 'REPORTER_ADDRESS2',
 'REPORTER_CITY',
 'REPORTER_STATE',
 'REPORTER_ZIP',
 'REPORTER_COUNTY',
 'BUYER_DEA_NO',
 'BUYER_BUS_ACT',
 'BUYER_NAME',
 'BUYER_ADDL_CO_INFO',
 'BUYER_ADDRESS1',
 'BUYER_ADDRESS2',
 'BUYER_CITY',
 'BUYER_STATE',
 'BUYER_ZIP',
 'BUYER_COUNTY',
 #'TRANSACTION_CODE',
 #'DRUG_CODE',
 #'NDC_NO',
 'DRUG_NAME',
 'QUANTITY',
 'UNIT',
 'ACTION_INDICATOR',
 #'ORDER_FORM_NO',
 #'CORRECTION_NO',
 'STRENGTH',
 'TRANSACTION_DATE',
 #'CALC_BASE_WT_IN_GM',
 'DOSAGE_UNIT',
 'TRANSACTION_ID',
 'Product_Name',
 'Ingredient_Name',
 'Measure',
 #'MME_Conversion_Factor',
 #'Combined_Labeler_Name',
 #'Revised_Company_Name',
 #'Reporter_family',
 #'dos_str'
]

In [12]:
data = dd.read_table(
    "s3://coiled-datasets/dea-opioid/arcos_washpost.tsv",
    dtype={
 'REPORTER_DEA_NO': 'object',
 'REPORTER_BUS_ACT': 'object',
 'REPORTER_NAME': 'object',
 'REPORTER_ADDL_CO_INFO': 'object',
 'REPORTER_ADDRESS1': 'object',
 'REPORTER_ADDRESS2': 'object',
 'REPORTER_CITY': 'object',
 'REPORTER_STATE': 'object',
 'REPORTER_ZIP': 'int64',
 'REPORTER_COUNTY': 'object',
 'BUYER_DEA_NO': 'object',
 'BUYER_BUS_ACT': 'object',
 'BUYER_NAME': 'object',
 'BUYER_ADDL_CO_INFO': 'object',
 'BUYER_ADDRESS1': 'object',
 'BUYER_ADDRESS2': 'object',
 'BUYER_CITY': 'object',
 'BUYER_STATE': 'object',
 'BUYER_ZIP': 'int64',
 'BUYER_COUNTY': 'object',
 'TRANSACTION_CODE': 'object',
 'DRUG_CODE': 'int64',
 'NDC_NO': 'object',
 'DRUG_NAME': 'object',
 'QUANTITY': 'float64',
 'UNIT': 'object',
 'ACTION_INDICATOR': 'object',
 'ORDER_FORM_NO': 'object',
 'CORRECTION_NO': 'float64',
 'STRENGTH': 'float64',
 'TRANSACTION_DATE': 'int64',
 'CALC_BASE_WT_IN_GM': 'float64',
 'DOSAGE_UNIT': 'float64',
 'TRANSACTION_ID': 'int64',
 'Product_Name': 'object',
 'Ingredient_Name': 'object',
 'Measure': 'object',
 'MME_Conversion_Factor': 'float64',
 'Combined_Labeler_Name': 'object',
 'Revised_Company_Name': 'object',
 'Reporter_family': 'object',
 'dos_str': 'float64'
    },
    usecols=columns_selection,
)

ddf = data.repartition(partition_size='100MB').persist()

In [13]:
ddf.head()

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,QUANTITY,UNIT,ACTION_INDICATOR,STRENGTH,TRANSACTION_DATE,DOSAGE_UNIT,TRANSACTION_ID,Product_Name,Ingredient_Name,Measure
0,PA0006836,DISTRIBUTOR,ACE SURGICAL SUPPLY CO INC,NaN,1034 PEARL STREET,NaN,BROCKTON,MA,2301,PLYMOUTH,...,1.0,NaN,NaN,NaN,12262012,100.0,64,HYDROCODONE BIT/ACETA 10MG/500MG USP,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB
1,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,4.0,NaN,NaN,NaN,3112009,40.0,52,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB
2,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,40.0,NaN,NaN,NaN,11252008,1200.0,119,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB
3,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,20.0,NaN,NaN,NaN,6122009,600.0,34,HYDROCODONEBITARTRATE & ACETA 7.5MG,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB
4,PA0021179,DISTRIBUTOR,APOTHECA INC,NaN,1622 N 16TH ST,NaN,PHOENIX,AZ,85006,MARICOPA,...,10.0,NaN,NaN,NaN,10022009,300.0,19,HYDROCODONE BITARTRATE & ACETA 5MG/,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB


In [8]:
%%time
ddf.groupby('REPORTER_CITY').DOSAGE_UNIT.mean().compute()

CPU times: user 545 ms, sys: 29.9 ms, total: 575 ms
Wall time: 20.6 s


REPORTER_CITY
BROCKTON              300.000000
COLUMBUS              237.618326
GOUVERNEUR            655.202560
GRANDVIEW HEIGHTS     797.290070
GUAYNABO              500.919179
                        ...     
EAST LIVERPOOL       5600.000000
SUGAR GROVE           100.000000
NEWTOWN              2800.000000
LINCOLN                 4.000000
NEW HYDE PARK           5.000000
Name: DOSAGE_UNIT, Length: 418, dtype: float64

In [21]:
ddf.dtypes.apply(lambda x: x.name).to_dict()

{'REPORTER_DEA_NO': 'object',
 'REPORTER_BUS_ACT': 'object',
 'REPORTER_NAME': 'object',
 'REPORTER_ADDL_CO_INFO': 'object',
 'REPORTER_ADDRESS1': 'object',
 'REPORTER_ADDRESS2': 'object',
 'REPORTER_CITY': 'object',
 'REPORTER_STATE': 'object',
 'REPORTER_ZIP': 'int64',
 'REPORTER_COUNTY': 'object',
 'BUYER_DEA_NO': 'object',
 'BUYER_BUS_ACT': 'object',
 'BUYER_NAME': 'object',
 'BUYER_ADDL_CO_INFO': 'object',
 'BUYER_ADDRESS1': 'object',
 'BUYER_ADDRESS2': 'object',
 'BUYER_CITY': 'object',
 'BUYER_STATE': 'object',
 'BUYER_ZIP': 'int64',
 'BUYER_COUNTY': 'object',
 'TRANSACTION_CODE': 'object',
 'DRUG_CODE': 'int64',
 'NDC_NO': 'object',
 'DRUG_NAME': 'object',
 'QUANTITY': 'float64',
 'UNIT': 'object',
 'ACTION_INDICATOR': 'object',
 'ORDER_FORM_NO': 'object',
 'CORRECTION_NO': 'float64',
 'STRENGTH': 'float64',
 'TRANSACTION_DATE': 'int64',
 'CALC_BASE_WT_IN_GM': 'float64',
 'DOSAGE_UNIT': 'float64',
 'TRANSACTION_ID': 'int64',
 'Product_Name': 'object',
 'Ingredient_Name': 'objec

In [16]:
ddf.columns.to_list()

['REPORTER_DEA_NO',
 'REPORTER_BUS_ACT',
 'REPORTER_NAME',
 'REPORTER_ADDL_CO_INFO',
 'REPORTER_ADDRESS1',
 'REPORTER_ADDRESS2',
 'REPORTER_CITY',
 'REPORTER_STATE',
 'REPORTER_ZIP',
 'REPORTER_COUNTY',
 'BUYER_DEA_NO',
 'BUYER_BUS_ACT',
 'BUYER_NAME',
 'BUYER_ADDL_CO_INFO',
 'BUYER_ADDRESS1',
 'BUYER_ADDRESS2',
 'BUYER_CITY',
 'BUYER_STATE',
 'BUYER_ZIP',
 'BUYER_COUNTY',
 'TRANSACTION_CODE',
 'DRUG_CODE',
 'NDC_NO',
 'DRUG_NAME',
 'QUANTITY',
 'UNIT',
 'ACTION_INDICATOR',
 'ORDER_FORM_NO',
 'CORRECTION_NO',
 'STRENGTH',
 'TRANSACTION_DATE',
 'CALC_BASE_WT_IN_GM',
 'DOSAGE_UNIT',
 'TRANSACTION_ID',
 'Product_Name',
 'Ingredient_Name',
 'Measure',
 'MME_Conversion_Factor',
 'Combined_Labeler_Name',
 'Revised_Company_Name',
 'Reporter_family',
 'dos_str']

In [9]:
ddf.npartitions

3750

In [10]:
from coiled import performance_report

with performance_report('to-parquet-report.html'):
    ddf.to_parquet(
        "s3://coiled-datasets/dea-opioid/arcos_washpost.parquet"
    )

/Users/rpelgrim/mambaforge/envs/coiled-base/lib/python3.8/site-packages/coiled/utils.py:396: ExperimentalFeatureWarning: performance_report is an experimental feature which is subject to breaking changes, being removed, or otherwise updated without notice and should be used accordingly.
  warnings.warn(


]8;id=191657;https://cloud.coiled.io/rrpelgrim/reports/730\Performance Report Available at: https://cloud.coiled.io/rrpelgrim/reports/730]8;;\

KeyboardInterrupt: 

In [ ]:
ddf_test = dd.read_parquetuet(
    "s3://coiled-datasets/dea-opioid/arcos_washpost.parquet")